In [1]:
import numpy as np
import pandas as pd
import itertools
import functools


# IPF - Higher dimensions (age, gender, having a partner)

In [2]:
# example adjusted from: https://datascience.oneoffcoder.com/ipf-ii.html
# I have started with age, gender and having a partner, if the approach looks promising, we can add the other variables

n/total: 15555

Age
- age_group_1 (24-34): 3320 - 21,3%
- age_group_2 (35-44): 3355 - 21,6%
- age_group_3 (45-54): 5132 - 33%
- age_group_4 (55-64): 3748 - 24,1%

Gender
- male: 6236 - 40,1%
- female: 9319 - 59,9%

partner
- yes: 12304 - 79,1%
- no: 3251 - 20,9%



In [3]:
# Set a seed for reproducibility
np.random.seed(42)

# Define means and standard deviations for weights (assumed values)
mean_weight = 150
std_dev_weight = 10

# Total population
total_population = 1555 # didn't manage to recreate the real n (15555) so I scaled to 1555

# Age groups
age_groups = ['24-34', '35-44', '45-54', '55-64']
age_counts = [int(3320/10), int(3355/10), int(5132/10), int(3748/10)]

# Gender groups
gender_groups = ['male', 'female']
gender_counts = [int(6236/10), int(9319/10)]

# Partner groups
partner_groups = ['yes', 'no']
partner_counts = [int(12304/10), int(3251/10)]

# Generate data
data = []

for age_group, age_count in zip(age_groups, age_counts):
    for gender_group, gender_count in zip(gender_groups, gender_counts):
        for partner_group, partner_count in zip(partner_groups, partner_counts):
            # Assume the intersection of age, gender, and partner status is evenly distributed
            n_individuals = int(age_count * gender_count * partner_count / total_population) 

            # Generate weights
            weights = np.random.normal(mean_weight, std_dev_weight, n_individuals)

            # Append to data
            data += [
                {'age': age_group, 'gender': gender_group, 'partner': partner_group, 'weight': w}
                for w in weights
            ]

# Create DataFrame
df = pd.DataFrame(data)


In [4]:
df.head()

,age,gender,partner,weight
0,24-34,male,yes,154.967142
1,24-34,male,yes,148.617357
2,24-34,male,yes,156.476885
3,24-34,male,yes,165.230299
4,24-34,male,yes,147.658466


In [6]:
# It's a pity that the algorithm I found works only with matrices -- this makes it hard to work with variables that have different levels

In [7]:
# Define new age groups in order to square the matrix
new_age_groups = {
    '24-34': '24-44',
    '35-44': '24-44',
    '45-54': '45-64',
    '55-64': '45-64',
}

# Map old age groups to new age groups
df['new_age'] = df['age'].map(new_age_groups)

# View the dataframe
print(df.head())


     age gender partner      weight new_age
0  24-34   male     yes  154.967142   24-44
1  24-34   male     yes  148.617357   24-44
2  24-34   male     yes  156.476885   24-44
3  24-34   male     yes  165.230299   24-44
4  24-34   male     yes  147.658466   24-44


Do they match my initial values? let's check

In [25]:
df

,age,gender,partner,weight,new_age
0,24-34,male,yes,154.967142,24-44
1,24-34,male,yes,148.617357,24-44
2,24-34,male,yes,156.476885,24-44
3,24-34,male,yes,165.230299,24-44
4,24-34,male,yes,147.658466,24-44
...,...,...,...,...,...
2414903,55-64,female,no,148.201249,45-64
2414904,55-64,female,no,130.373599,45-64
2414905,55-64,female,no,143.137656,45-64
2414906,55-64,female,no,155.517071,45-64


In [8]:
df.age.value_counts().sort_index() / df.shape[0]


24-34    0.213642
35-44    0.215573
45-54    0.330116
55-64    0.240669
Name: age, dtype: float64

In [9]:
df.gender.value_counts().sort_index() / df.shape[0]


female    0.599099
male      0.400901
Name: gender, dtype: float64

In [10]:
df.partner.value_counts().sort_index() / df.shape[0]


no     0.209003
yes    0.790997
Name: partner, dtype: float64

yes they do

## Contingency table

In [11]:
def get_target_marginals(d):
    factors = list(d.keys())
    targets = [sorted([(k2, v2) for k2, v2 in v.items()]) for k, v in d.items()]
    targets = np.array([[v for _, v in item] for item in targets])
    return factors, targets

def get_table(df, targets):
    factors, target_marginals = get_target_marginals(targets)

    cross_tab = pd.crosstab(df[factors[0]], [df[c] for c in factors[1:]])
    shape = tuple([df[c].unique().shape[0] for c in factors])
    table = cross_tab.values.reshape(shape)

    return factors, target_marginals, table


In [12]:
f, u, X = get_table(df, {
    'new_age': {'24-44': int(6675/10), '45-64': int(8880/10)},
    'gender': {'male': int(6236/10), 'female': int(9319/10)},
    'partner': {'yes': int(12304/10), 'no': int(3251/10)}
})

In [13]:
X

array([[[129785, 491190],
        [ 86849, 328690]],

       [[172593, 653202],
        [115495, 437104]]])

In [14]:
u

array([[ 667,  888],
       [ 931,  623],
       [ 325, 1230]])

In [15]:
f

['new_age', 'gender', 'partner']

In [16]:
def get_coordinates(M):
    return list(itertools.product(*[list(range(n)) for n in M.shape]))

def get_marginals(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c]) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: sum([v[1] for v in g]) for k, g in counts}

    return counts

def get_all_marginals(M):
    return np.array([[v for _, v in get_marginals(M, i).items()]
                     for i in range(len(M.shape))])

def get_counts(M, i):
    coordinates = get_coordinates(M)

    key = lambda tup: tup[0]
    counts = [(c[i], M[c], c) for c in coordinates]
    counts = sorted(counts, key=key)
    counts = itertools.groupby(counts, key=key)
    counts = {k: [(tup[1], tup[2]) for tup in g] for k, g in counts}

    return counts

def update_values(M, i, u):
    marg = get_marginals(M, i)
    vals = get_counts(M, i)

    d = [[(c, n * u[k] / marg[k]) for n, c in v] for k, v in vals.items()]
    d = itertools.chain(*d)
    d = list(d)

    return d

def ipf_update(M, u):
    for i in range(len(M.shape)):
        values = update_values(M, i, u[i])
        for idx, v in values:
            M[idx] = v

    o = get_all_marginals(M)
    d = get_deltas(o, u)

    return M, d

def get_deltas(o, t):
    return np.array([np.linalg.norm(o[r] - t[r], 2) for r in range(o.shape[0])])

def get_weights(X, max_iters=50, zero_threshold=0.0001, convergence_threshold=3, debug=True):
    M = X.copy()

    d_prev = np.zeros(len(M.shape))
    count_zero = 0

    for _ in range(max_iters):
        M, d_next = ipf_update(M, u)
        d = np.linalg.norm(d_prev - d_next, 2)

        if d < zero_threshold:
            count_zero += 1

        if debug:
            print(','.join([f'{v:.5f}' for v in d_next]), d)
        d_prev = d_next

        if count_zero >= convergence_threshold:
            break

    w = M / M.sum()
    return w

In [17]:
w = get_weights(X)

2.82843,2.23607,2.82843 4.582575694955841
2.82843,2.23607,2.82843 0.0
2.82843,2.23607,2.82843 0.0
2.82843,2.23607,2.82843 0.0


In [18]:
w

array([[[0.05351386, 0.20373952],
        [0.03546099, 0.13604126]],

       [[0.07156673, 0.27079304],
        [0.04771115, 0.18117344]]])

In [20]:
# sample (with replacement) from the synthetic data.

In [ ]:
def get_sampling_weights(df, f, w):
    get_filters = lambda df, fields, values: [df[f] == v for f, v in zip(fields, values)]
    get_total = lambda df, fields, values: df[functools.reduce(lambda a, b: a & b, get_filters(df, fields, values))].shape[0]

    return {k: v / get_total(df, f, k) for k, v in zip(list(itertools.product(*[sorted(df[c].unique()) for c in f])), np.ravel(w))}

def get_samples(df, f, w, n=10_000):
    weights = get_sampling_weights(df, f, w)
    s = df.apply(lambda r: weights[tuple([r[c] for c in f])], axis=1)
    return df.sample(n=n, replace=True, weights=s)

This is the new population based on the weights learned from the initial distribution:

In [22]:
sample_df = get_samples(df, f, w)
sample_df

,age,gender,partner,weight,new_age
218209,24-34,female,yes,126.902369,24-44
1268498,45-54,male,yes,162.689874,45-64
605364,35-44,male,yes,140.007049,24-44
2325557,55-64,female,yes,153.120736,45-64
1612844,45-54,female,yes,147.695474,45-64
...,...,...,...,...,...
286322,24-34,female,yes,136.339755,24-44
1682211,45-54,female,yes,146.328778,45-64
401180,24-34,female,yes,133.728617,24-44
2203527,55-64,female,yes,148.408483,45-64


In [23]:
# Here is the cross-tabulation of the resulting sampled matrix -- to double check that they match our initial values

In [24]:
ct = pd.crosstab(sample_df.partner, [sample_df.new_age, sample_df.gender])
ct

new_age  24-44        45-64      
gender  female  male female  male
partner                          
no         521   367    708   491
yes       2030  1319   2774  1790

In [68]:
sample_df.age.value_counts().sort_index() / sample_df.shape[0]

24-34    0.2101
35-44    0.2136
45-54    0.3378
55-64    0.2385
Name: age, dtype: float64

In [72]:
sample_df.gender.value_counts().sort_index() / sample_df.shape[0]

female    0.6033
male      0.3967
Name: gender, dtype: float64

In [71]:
sample_df.partner.value_counts().sort_index() / sample_df.shape[0]

no     0.2087
yes    0.7913
Name: partner, dtype: float64